<a href="https://colab.research.google.com/github/nmatsumoto-lgtm/study-KIKAGAKU/blob/main/%E4%B8%8D%E5%8B%95%E7%94%A3%E4%BE%A1%E6%A0%BC%E4%BA%88%E6%B8%AC%E3%82%A2%E3%83%97%E3%83%AA_Ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import torch
import torch.nn as nn
import re
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from datetime import datetime
from pathlib import Path

In [ ]:
df = pd.read_csv("Tokyo_Chuo Ward_20242_20251.csv", encoding="cp932")

In [ ]:
df.head(10)

,種類,価格情報区分,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,間取り,...,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時期,改装,取引の事情等
0,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),3.0,140000000,３ＬＤＫ,...,2001年,ＳＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
1,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),3.0,26000000,２Ｋ,...,1979年,ＳＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
2,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),3.0,56000000,１ＬＤＫ,...,2007年,ＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
3,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),1.0,47000000,１ＬＤＫ,...,1998年,ＳＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
4,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),3.0,58000000,１ＬＤＫ,...,2007年,ＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
5,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),1.0,160000000,３ＬＤＫ,...,2015年,ＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
6,中古マンション等,成約価格情報,13102,東京都,中央区,新富,新富町(東京),3.0,15000000,１Ｒ,...,1980年,NaN,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
7,中古マンション等,成約価格情報,13102,東京都,中央区,日本橋蛎殻町,水天宮前,3.0,23000000,１Ｋ,...,2001年,ＲＣ,NaN,NaN,NaN,NaN,NaN,2024年第2四半期,NaN,NaN
8,中古マンション等,成約価格情報,13102,東京都,中央区,日本橋蛎殻町,水天宮前,4.0,170000000,３ＬＤＫ,...,2015年,ＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN
9,中古マンション等,成約価格情報,13102,東京都,中央区,日本橋蛎殻町,水天宮前,3.0,72000000,１ＬＤＫ,...,2000年,ＳＲＣ,NaN,NaN,商業,NaN,NaN,2024年第2四半期,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   種類         1790 non-null   object 
 1   価格情報区分     1790 non-null   object 
 2   市区町村コード    1790 non-null   int64  
 3   都道府県名      1790 non-null   object 
 4   市区町村名      1790 non-null   object 
 5   地区名        1790 non-null   object 
 6   最寄駅：名称     1776 non-null   object 
 7   最寄駅：距離（分）  1771 non-null   float64
 8   取引価格（総額）   1790 non-null   int64  
 9   間取り        1757 non-null   object 
 10  面積（㎡）      1790 non-null   int64  
 11  建築年        1752 non-null   object 
 12  建物の構造      1743 non-null   object 
 13  用途         432 non-null    object 
 14  今後の利用目的    538 non-null    object 
 15  都市計画       1639 non-null   object 
 16  建ぺい率（％）    549 non-null    float64
 17  容積率（％）     549 non-null    float64
 18  取引時期       1790 non-null   object 
 19  改装         541 non-null    object 
 20  取引の事情等  

#前処理

In [ ]:
RAW = Path("data/raw")
OUT = Path("data/interim"); OUT.mkdir(parents=True, exist_ok=True)

In [ ]:
# 必要な列だけ抽出
df = df[[
    "最寄駅：名称", "最寄駅：距離（分）", "間取り", "面積（㎡）",
    "建築年", "取引価格（総額）"
]].copy()

In [ ]:
# 築年数の計算
def to_year(x):
    if pd.isna(x): return np.nan
    m = re.search(r"(\d{4})年", str(x))
    return int(m.group(1)) if m else np.nan

In [ ]:
build_year = df["建築年"].apply(to_year)
current_year = datetime.now().year
df["築年数"] = (current_year - build_year).clip(lower=0)

In [ ]:
# 数値化＆欠損処理
for col in ["最寄駅：距離（分）", "面積（㎡）", "築年数"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col] = df[col].fillna(df[col].median())

In [ ]:
# 間取りの正規化
def normalize_layout(s: str):
    if not isinstance(s, str): return "その他"
    s = s.upper().replace("Ｌ","L").replace("Ｄ","D").replace("Ｋ","K").replace("Ｒ","R")
    m = re.search(r"(\d+)(LDK|DK|K|R)", s)
    return m.group(0) if m else "その他"

In [ ]:
df["layout"] = df["間取り"].astype(str).apply(normalize_layout)

In [ ]:
# 最終的な学習用テーブル
train = df.rename(columns={
    "最寄駅：名称": "station",
    "最寄駅：距離（分）": "walk_min",
    "面積（㎡）": "area_sqm",
    "取引価格（総額）": "price_yen"
})[["station","walk_min","築年数","layout","area_sqm","price_yen"]]

In [ ]:
# 保存
OUT = Path("data/interim"); OUT.mkdir(parents=True, exist_ok=True)
train.to_csv(OUT / "train_tabular.csv", index=False)
print("Saved:", OUT / "train_tabular.csv")

Saved: data/interim/train_tabular.csv


In [ ]:
# # アプリ用の駅リスト
# stations = sorted(train["station"].dropna().unique())
# pd.Series(stations).to_csv(OUT / "stations.csv", index=False, header=False)

In [ ]:
s = (df["最寄駅：名称"]
       .astype(str)
       .replace({"nan": np.nan, "None": np.nan, "": np.nan})
       .dropna()
       .str.normalize("NFKC")                  # 全角→半角など
       .str.replace("（", "(", regex=False)    # 全角カッコ→半角
       .str.replace("）", ")", regex=False)
       .str.replace("　", " ", regex=False)    # 全角スペース→半角
       .str.strip()
       .str.replace(r"\s+", " ", regex=True))  # 連続空白の圧縮

In [ ]:
stations = sorted(s.unique().tolist())

Path("data/interim").mkdir(parents=True, exist_ok=True)
# ★ Windows/Excelでも化けないように「utf-8-sig」で保存
pd.Series(stations).to_csv("data/interim/stations.csv",
                           index=False, header=False, encoding="utf-8-sig")

print("stations.csv 件数:", len(stations))
print(stations[:20])

stations.csv 件数: 25
['三越前', '京橋(東京)', '人形町', '八丁堀(東京)', '勝どき', '宝町(東京)', '小伝馬町', '新富町(東京)', '新日本橋', '新橋', '日本橋(東京)', '月島', '東京', '東日本橋', '東銀座', '水天宮前', '汐留', '浜町', '築地', '築地市場']


#学習：train_baseline.py（CV＋分位点＝信頼区間つき）

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_log_error
from joblib import dump

In [ ]:
IN = Path('data/interim/train_tabular.csv')
MODEL_DIR = Path('models'); MODEL_DIR.mkdir(exist_ok=True, parents=True)

In [ ]:
df = pd.read_csv(IN)

In [ ]:
feat_cat = ['station', 'layout']
feat_num = ['walk_min', '築年数', 'area_sqm']

In [ ]:
X_cat = df[feat_cat].fillna('NA')
X_num = df[feat_num]
y = df['price_yen'].values

In [ ]:
# カテゴリは序数エンコード（簡単）
oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_cat_enc = oe.fit_transform(X_cat)
X = np.hstack([X_cat_enc, X_num.values])

In [ ]:
# 5-fold CV
kf = KFold(n_splits=5, shuffle=True, random_state=42)
maes, rmsles = [], []
best_iters = []

In [ ]:
params = dict(
    objective='regression',
    metric='l1',
    learning_rate=0.05,
    num_leaves=63,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=1,
    verbose=-1,
    seed=42,
)

In [ ]:
for fold, (tr, va) in enumerate(kf.split(X)):
    X_tr, X_va = X[tr], X[va]
    y_tr, y_va = y[tr], y[va]

    dtr = lgb.Dataset(X_tr, label=y_tr)
    dva = lgb.Dataset(X_va, label=y_va)

    model = lgb.train(params, dtr, valid_sets=[dva],
                      num_boost_round=2000,
                      callbacks=[lgb.early_stopping(100)])

    pred = model.predict(X_va, num_iteration=model.best_iteration)
    mae = mean_absolute_error(y_va, pred)
    rmsle = np.sqrt(mean_squared_log_error(y_va.clip(min=1), pred.clip(min=1)))
    maes.append(mae); rmsles.append(rmsle)
    best_iters.append(model.best_iteration)
    print(f"Fold {fold}: MAE={mae:,.0f}, RMSLE={rmsle:.4f}")

print(f"CV MAE: {np.mean(maes):,.0f} ± {np.std(maes):,.0f}")
print(f"CV RMSLE: {np.mean(rmsles):.4f} ± {np.std(rmsles):.4f}")

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[382]	valid_0's l1: 1.20396e+07
Fold 0: MAE=12,039,631, RMSLE=0.1816
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[189]	valid_0's l1: 1.20705e+07
Fold 1: MAE=12,070,542, RMSLE=0.1707
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[357]	valid_0's l1: 1.41239e+07
Fold 2: MAE=14,123,895, RMSLE=0.1816
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[307]	valid_0's l1: 1.24812e+07
Fold 3: MAE=12,481,249, RMSLE=0.1955
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[170]	valid_0's l1: 1.19665e+07
Fold 4: MAE=11,966,545, RMSLE=0.1752
CV MAE: 12,536,372 ± 813,845
CV RMSLE: 0.1809 ± 0.0084


In [ ]:
# 全データで最終モデル学習（CVの平均best_iterationを採用）
best_round = int(np.mean(best_iters))
final_model = lgb.train(params, lgb.Dataset(X, label=y), num_boost_round=best_round)

In [ ]:
# 信頼区間用（分位点回帰 q10/q90）
for alpha, name in [(0.1, 'baseline_q10'), (0.9, 'baseline_q90')]:
    q_params = params.copy(); q_params.update(dict(objective='quantile', alpha=alpha))
    q_model = lgb.train(q_params, lgb.Dataset(X, label=y), num_boost_round=best_round)
    dump({'model': q_model, 'encoder': oe, 'feat_cat': feat_cat, 'feat_num': feat_num},
         f"models/{name}.pkl")

In [ ]:
# 保存
from joblib import dump
dump({'model': final_model, 'encoder': oe, 'feat_cat': feat_cat, 'feat_num': feat_num},
     'models/baseline_lgbm.pkl')
print('Saved models → models/')

Saved models → models/


In [ ]:
from google.colab import files
files.download("data/interim/train_tabular.csv")
files.download("data/interim/stations.csv")
files.download("models/baseline_lgbm.pkl")
files.download("models/baseline_q10.pkl")
files.download("models/baseline_q90.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>